# Healthcare Chatbot Model Training

This notebook trains a comprehensive healthcare chatbot with the following capabilities:
- **Disease Diagnosis**: Analyze symptoms and provide preliminary diagnosis
- **Treatment Recommendations**: Suggest treatment options based on conditions
- **Appointment Booking**: Schedule medical appointments with doctors
- **Appointment Tracking**: Check existing appointments and medical history
- **Health Education**: Provide health tips and preventive care information
- **Emergency Detection**: Identify emergency situations and provide immediate guidance

## Features:
✅ Multi-intent classification
✅ Symptom-to-disease mapping
✅ Treatment recommendations
✅ Appointment management
✅ Emergency detection
✅ Health education content

In [13]:
# Import required libraries
import json
import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

# Natural Language Processing
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

# Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

# Additional utilities
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ All libraries imported successfully!")
print(f"Training started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ All libraries imported successfully!
Training started at: 2025-05-28 21:00:54


In [ ]:
# Download required NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    print("✅ NLTK data downloaded successfully!")
except:
    print("⚠️ NLTK data download failed, but continuing...")

# Initialize preprocessing tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stop_words.update(['xin', 'chào', 'tôi', 'bạn', 'của', 'và', 'có', 'là', 'không', 'với', 'cho', 'để'])

print("✅ Text preprocessing tools initialized!")

In [ ]:
# Load healthcare knowledge base
with open('healthcare_knowledge_base.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extract patterns and labels for training
patterns = []
labels = []
intent_responses = {}

for intent in data['intents']:
    tag = intent['tag']
    intent_responses[tag] = intent['responses']
    
    for pattern in intent['patterns']:
        patterns.append(pattern)
        labels.append(tag)

print(f"📊 Dataset Statistics:")
print(f"   Total patterns: {len(patterns)}")
print(f"   Total intents: {len(set(labels))}")
print(f"   Intent distribution:")

# Show intent distribution
label_counts = pd.Series(labels).value_counts()
for intent, count in label_counts.items():
    print(f"     {intent}: {count} patterns")

print("\n✅ Knowledge base loaded successfully!")

In [ ]:
# Advanced text preprocessing function
def preprocess_text(text):
    """
    Comprehensive text preprocessing for Vietnamese and English
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords and short words
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    
    # Lemmatize (for English words)
    try:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    except:
        pass
    
    return ' '.join(tokens)

# Test preprocessing
test_text = "Tôi bị sốt cao và đau đầu, cần khám bác sĩ"
processed = preprocess_text(test_text)
print(f"Original: {test_text}")
print(f"Processed: {processed}")

# Preprocess all patterns
processed_patterns = [preprocess_text(pattern) for pattern in patterns]
print(f"\n✅ Text preprocessing completed!")
print(f"Sample processed patterns:")
for i in range(min(3, len(processed_patterns))):
    print(f"   {i+1}. {processed_patterns[i]}")

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    processed_patterns, labels, 
    test_size=0.2, 
    random_state=42, 
    stratify=labels
)

print(f"📊 Data Split:")
print(f"   Training samples: {len(X_train)}")
print(f"   Testing samples: {len(X_test)}")

# Create TF-IDF vectorizer with optimized parameters
vectorizer = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2),  # Include bigrams
    min_df=1,
    max_df=0.9,
    stop_words=None  # We already removed stopwords
)

# Fit and transform training data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"\n✅ TF-IDF features created!")
print(f"   Feature matrix shape: {X_train_tfidf.shape}")
print(f"   Vocabulary size: {len(vectorizer.vocabulary_)}")

In [ ]:
# Train multiple models for comparison
models = {
    'Naive Bayes': MultinomialNB(alpha=0.1),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': SVC(kernel='linear', probability=True, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

model_scores = {}
best_model = None
best_score = 0

print("🚀 Training multiple models...")
for name, model in models.items():
    print(f"\n   Training {name}...")
    
    # Train model
    model.fit(X_train_tfidf, y_train)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train_tfidf, y_train, cv=3, scoring='accuracy')
    avg_cv_score = np.mean(cv_scores)
    
    # Test accuracy
    test_pred = model.predict(X_test_tfidf)
    test_accuracy = accuracy_score(y_test, test_pred)
    
    model_scores[name] = {
        'cv_score': avg_cv_score,
        'test_accuracy': test_accuracy,
        'model': model
    }
    
    print(f"     Cross-validation score: {avg_cv_score:.4f}")
    print(f"     Test accuracy: {test_accuracy:.4f}")
    
    # Track best model
    if avg_cv_score > best_score:
        best_score = avg_cv_score
        best_model = model
        best_model_name = name

print(f"\n🏆 Best Model: {best_model_name} (CV Score: {best_score:.4f})")

In [ ]:
# Detailed evaluation of best model
print(f"\n📈 Detailed Evaluation - {best_model_name}:")

# Predictions
y_pred = best_model.predict(X_test_tfidf)
y_pred_proba = best_model.predict_proba(X_test_tfidf)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
labels_unique = sorted(set(labels))

plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels_unique, yticklabels=labels_unique)
plt.title(f'Confusion Matrix - {best_model_name}')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Feature importance (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    feature_names = vectorizer.get_feature_names_out()
    importances = best_model.feature_importances_
    
    # Get top features
    indices = np.argsort(importances)[::-1][:20]
    top_features = [(feature_names[i], importances[i]) for i in indices]
    
    print("\nTop 10 Important Features:")
    for i, (feature, importance) in enumerate(top_features[:10]):
        print(f"   {i+1}. {feature}: {importance:.4f}")

print("\n✅ Model evaluation completed!")

In [ ]:
# Healthcare Chatbot Class with Advanced Features
class HealthcareChatbot:
    def __init__(self, model, vectorizer, intent_responses):
        self.model = model
        self.vectorizer = vectorizer
        self.intent_responses = intent_responses
        self.conversation_history = []
        self.user_context = {}
        
    def preprocess_input(self, text):
        """Preprocess user input"""
        return preprocess_text(text)
    
    def predict_intent(self, text):
        """Predict intent with confidence score"""
        processed_text = self.preprocess_input(text)
        text_tfidf = self.vectorizer.transform([processed_text])
        
        # Get prediction and probabilities
        intent = self.model.predict(text_tfidf)[0]
        probabilities = self.model.predict_proba(text_tfidf)[0]
        confidence = max(probabilities)
        
        return intent, confidence
    
    def get_response(self, user_input):
        """Generate response based on user input"""
        intent, confidence = self.predict_intent(user_input)
        
        # Log conversation
        self.conversation_history.append({
            'user_input': user_input,
            'predicted_intent': intent,
            'confidence': confidence,
            'timestamp': datetime.now().isoformat()
        })
        
        # Handle low confidence
        if confidence < 0.5:
            return {
                'intent': 'unknown',
                'confidence': confidence,
                'response': "Xin lỗi, tôi không hiểu rõ ý bạn. Bạn có thể nói rõ hơn về triệu chứng hoặc vấn đề sức khỏe của mình không? Hoặc bạn có thể hỏi về: chẩn đoán triệu chứng, đặt lịch khám, tư vấn sức khỏe.",
                'suggestions': ['Tôi bị đau đầu', 'Đặt lịch khám bệnh', 'Tư vấn sức khỏe']
            }
        
        # Get response for intent
        responses = self.intent_responses.get(intent, ["Xin lỗi, tôi chưa được train để xử lý câu hỏi này."])
        response = np.random.choice(responses)
        
        # Special handling for appointment booking
        if intent == 'appointment_booking':
            return self.handle_appointment_booking(response)
        
        # Special handling for emergency
        elif intent == 'emergency':
            return self.handle_emergency(response)
        
        # Special handling for symptom diagnosis
        elif intent in ['fever_symptoms', 'headache_symptoms', 'cough_symptoms', 'stomach_pain']:
            return self.handle_symptom_diagnosis(intent, response)
        
        return {
            'intent': intent,
            'confidence': confidence,
            'response': response,
            'additional_info': self.get_additional_info(intent)
        }
    
    def handle_appointment_booking(self, base_response):
        """Handle appointment booking with interactive flow"""
        return {
            'intent': 'appointment_booking',
            'response': base_response,
            'action': 'collect_appointment_info',
            'required_fields': ['name', 'phone', 'date_of_birth', 'symptoms', 'department', 'preferred_time'],
            'next_step': 'Hãy bắt đầu bằng cách cho tôi biết họ tên đầy đủ của bạn?'
        }
    
    def handle_emergency(self, base_response):
        """Handle emergency situations with immediate action"""
        return {
            'intent': 'emergency',
            'response': base_response,
            'priority': 'URGENT',
            'action': 'emergency_protocol',
            'immediate_steps': [
                'Gọi cấp cứu 115 ngay lập tức',
                'Giữ bình tĩnh và không di chuyển',
                'Chuẩn bị thông tin cơ bản: tuổi, triệu chứng, thời gian bắt đầu'
            ]
        }
    
    def handle_symptom_diagnosis(self, intent, base_response):
        """Enhanced symptom diagnosis with follow-up questions"""
        follow_up_questions = {
            'fever_symptoms': [
                'Nhiệt độ cơ thể hiện tại của bạn là bao nhiêu?',
                'Bạn có kèm theo triệu chứng nào khác không? (đau đầu, ho, đau họng)',
                'Triệu chứng đã kéo dài bao lâu?'
            ],
            'headache_symptoms': [
                'Đau đầu ở vị trí nào? (trán, thái dương, gáy)',
                'Mức độ đau từ 1-10 là bao nhiêu?',
                'Có kèm buồn nôn hoặc nhạy cảm với ánh sáng không?'
            ],
            'cough_symptoms': [
                'Ho khan hay có đờm?',
                'Ho nhiều vào thời gian nào trong ngày?',
                'Có kèm sốt hoặc khó thở không?'
            ],
            'stomach_pain': [
                'Đau bụng ở vị trí nào cụ thể?',
                'Mức độ đau và có đau liên tục không?',
                'Có kèm buồn nôn, tiêu chảy hoặc táo bón không?'
            ]
        }
        
        return {
            'intent': intent,
            'response': base_response,
            'follow_up_questions': follow_up_questions.get(intent, []),
            'recommended_action': 'Nếu triệu chứng nghiêm trọng hoặc kéo dài, hãy đặt lịch khám bác sĩ.',
            'appointment_suggestion': True
        }
    
    def get_additional_info(self, intent):
        """Provide additional helpful information based on intent"""
        additional_info = {
            'greeting': {
                'services': ['Chẩn đoán triệu chứng', 'Đặt lịch khám', 'Tư vấn sức khỏe', 'Thông tin thuốc'],
                'emergency_note': 'Trong trường hợp khẩn cấp, hãy gọi 115'
            },
            'health_tips': {
                'related_topics': ['Dinh dưỡng', 'Tập thể dục', 'Giấc ngủ', 'Quản lý stress'],
                'check_up_reminder': 'Nhớ khám sức khỏe định kỳ 6 tháng/lần'
            },
            'medicine_info': {
                'safety_warning': 'Luôn tham khảo bác sĩ hoặc dược sĩ trước khi dùng thuốc',
                'pharmacy_hotline': 'Hotline dược sĩ: 1900-xxxx'
            }
        }
        
        return additional_info.get(intent, {})
    
    def get_conversation_summary(self):
        """Get conversation history summary"""
        if not self.conversation_history:
            return "Chưa có cuộc trò chuyện nào."
        
        intents = [conv['predicted_intent'] for conv in self.conversation_history]
        intent_counts = pd.Series(intents).value_counts().to_dict()
        
        return {
            'total_interactions': len(self.conversation_history),
            'intent_distribution': intent_counts,
            'last_interaction': self.conversation_history[-1]['timestamp']
        }

print("✅ Healthcare Chatbot class created successfully!")

In [ ]:
# Initialize the Healthcare Chatbot
chatbot = HealthcareChatbot(best_model, vectorizer, intent_responses)

print("🤖 Healthcare Chatbot initialized successfully!")
print("\n" + "="*60)
print("             CHATBOT TESTING")
print("="*60)

# Test conversations
test_inputs = [
    "Xin chào, tôi cần tư vấn sức khỏe",
    "Tôi bị sốt cao và đau đầu từ 2 ngày nay",
    "Làm thế nào để đặt lịch khám bác sĩ?",
    "Tôi ho khan kéo dài, có đờm màu vàng",
    "Đau bụng dưới bên phải rất dữ dội",
    "Cho tôi một số lời khuyên để sống khỏe mạnh",
    "Cảm ơn bạn rất nhiều"
]

for i, user_input in enumerate(test_inputs, 1):
    print(f"\n[Cuộc hội thoại {i}]")
    print(f"👤 Người dùng: {user_input}")
    
    response = chatbot.get_response(user_input)
    
    print(f"🤖 Chatbot: {response['response']}")
    print(f"📊 Intent: {response['intent']} (Confidence: {response.get('confidence', 0):.2f})")
    
    # Show additional information if available
    if 'follow_up_questions' in response and response['follow_up_questions']:
        print("❓ Câu hỏi thêm:")
        for q in response['follow_up_questions'][:2]:  # Show first 2 questions
            print(f"   • {q}")
    
    if 'immediate_steps' in response:
        print("🚨 Các bước khẩn cấp:")
        for step in response['immediate_steps']:
            print(f"   • {step}")
    
    print("-" * 50)

# Show conversation summary
print("\n📈 Conversation Summary:")
summary = chatbot.get_conversation_summary()
print(f"   Total interactions: {summary['total_interactions']}")
print(f"   Intent distribution: {summary['intent_distribution']}")

print("\n✅ Chatbot testing completed successfully!")

In [ ]:
# Save the trained model and components
print("💾 Saving trained model and components...")

# Create model package
model_package = {
    'model': best_model,
    'vectorizer': vectorizer,
    'intent_responses': intent_responses,
    'model_name': best_model_name,
    'training_accuracy': best_score,
    'training_date': datetime.now().isoformat(),
    'features_count': len(vectorizer.vocabulary_),
    'intents_count': len(set(labels))
}

# Save with pickle
with open('healthcare_chatbot_model.pkl', 'wb') as f:
    pickle.dump(model_package, f)

print("✅ Model saved as 'healthcare_chatbot_model.pkl'")

# Save model metadata
model_metadata = {
    'model_type': best_model_name,
    'training_accuracy': float(best_score),
    'test_accuracy': float(model_scores[best_model_name]['test_accuracy']),
    'training_date': datetime.now().isoformat(),
    'total_patterns': len(patterns),
    'total_intents': len(set(labels)),
    'features_count': len(vectorizer.vocabulary_),
    'model_parameters': str(best_model.get_params()),
    'intent_list': list(set(labels))
}

with open('model_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(model_metadata, f, ensure_ascii=False, indent=2)

print("✅ Model metadata saved as 'model_metadata.json'")

# Create simple deployment script
deployment_script = '''
#!/usr/bin/env python3
# Healthcare Chatbot Deployment Script

import pickle
import json
from datetime import datetime

class HealthcareChatbotLoader:
    def __init__(self, model_path='healthcare_chatbot_model.pkl'):
        """Load trained healthcare chatbot model"""
        with open(model_path, 'rb') as f:
            self.model_package = pickle.load(f)
        
        self.model = self.model_package['model']
        self.vectorizer = self.model_package['vectorizer']
        self.intent_responses = self.model_package['intent_responses']
        
        print(f"✅ Healthcare Chatbot loaded successfully!")
        print(f"   Model: {self.model_package['model_name']}")
        print(f"   Training Accuracy: {self.model_package['training_accuracy']:.2f}")
        print(f"   Trained on: {self.model_package['training_date']}")
    
    def predict(self, user_input):
        """Make prediction for user input"""
        # Preprocess text (simplified version)
        processed_text = user_input.lower()
        text_tfidf = self.vectorizer.transform([processed_text])
        
        # Predict
        intent = self.model.predict(text_tfidf)[0]
        confidence = max(self.model.predict_proba(text_tfidf)[0])
        
        # Get response
        responses = self.intent_responses.get(intent, ["Sorry, I don't understand."])
        response = responses[0]  # Take first response
        
        return {
            'intent': intent,
            'confidence': float(confidence),
            'response': response,
            'timestamp': datetime.now().isoformat()
        }

# Example usage
if __name__ == "__main__":
    # Load chatbot
    chatbot = HealthcareChatbotLoader()
    
    # Test
    test_inputs = [
        "Tôi bị sốt cao",
        "Đặt lịch khám bệnh",
        "Cảm ơn bạn"
    ]
    
    for user_input in test_inputs:
        result = chatbot.predict(user_input)
        print(f"Input: {user_input}")
        print(f"Intent: {result['intent']} (Confidence: {result['confidence']:.2f})")
        print(f"Response: {result['response'][:100]}...")
        print("-" * 50)
'''

with open('deploy_chatbot.py', 'w', encoding='utf-8') as f:
    f.write(deployment_script)

print("✅ Deployment script saved as 'deploy_chatbot.py'")

print("\n🎉 MODEL TRAINING COMPLETED SUCCESSFULLY!")
print("\n📋 Summary:")
print(f"   🏆 Best Model: {best_model_name}")
print(f"   📊 Training Accuracy: {best_score:.4f}")
print(f"   📊 Test Accuracy: {model_scores[best_model_name]['test_accuracy']:.4f}")
print(f"   📦 Total Patterns: {len(patterns)}")
print(f"   🎯 Total Intents: {len(set(labels))}")
print(f"   🔤 Vocabulary Size: {len(vectorizer.vocabulary_)}")
print(f"\n📁 Generated Files:")
print(f"   • healthcare_chatbot_model.pkl - Trained model")
print(f"   • model_metadata.json - Model information")
print(f"   • deploy_chatbot.py - Deployment script")
print(f"\n🚀 Ready for integration with your healthcare system!")

In [ ]:
# Create comprehensive healthcare knowledge base
healthcare_knowledge_base = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "xin chào", "chào bạn", "hello", "hi", "chào", "xin chào bác sĩ",
                "chào buổi sáng", "chào buổi chiều", "chào buổi tối", "bạn có khỏe không",
                "chào em", "chào anh", "chào chị", "xin chào!", "hi there", "hey"
            ],
            "responses": [
                "Xin chào! Tôi là trợ lý y tế AI. Tôi có thể giúp bạn:
✅ Tư vấn sơ bộ về triệu chứng
✅ Đặt lịch khám bệnh
✅ Tra cứu lịch hẹn
✅ Cung cấp thông tin y tế
Bạn cần hỗ trợ gì?",
                "Chào bạn! Tôi là chatbot y tế thông minh. Hãy mô tả triệu chứng hoặc nói 'đặt lịch khám' để tôi hỗ trợ bạn.",
                "Xin chào! Tôi sẵn sàng hỗ trợ bạn về các vấn đề sức khỏe. Bạn có triệu chứng gì cần tư vấn không?"
            ]
        },
        {
            "tag": "goodbye",
            "patterns": [
                "tạm biệt", "bye", "goodbye", "see you", "chào tạm biệt", "hẹn gặp lại",
                "cảm ơn và tạm biệt", "bye bye", "chúc ngủ ngon", "hẹn gặp lại", "tạm biệt nhé"
            ],
            "responses": [
                "Tạm biệt! Chúc bạn sức khỏe tốt. Hãy đến bác sĩ nếu triệu chứng không cải thiện.",
                "Chào tạm biệt! Luôn chăm sóc sức khỏe bản thân nhé.",
                "Hẹn gặp lại! Nhớ uống đủ nước và nghỉ ngơi hợp lý."
            ]
        },
        {
            "tag": "thanks",
            "patterns": [
                "cảm ơn", "thank you", "thanks", "cảm ơn bạn", "cảm ơn nhiều",
                "cảm ơn bác sĩ", "thanks a lot", "cảm ơn rất nhiều", "tôi cảm ơn"
            ],
            "responses": [
                "Không có gì! Tôi luôn sẵn sàng hỗ trợ bạn về các vấn đề sức khỏe.",
                "Rất vui được giúp đỡ bạn! Nhớ tái khám nếu cần thiết.",
                "Đó là nhiệm vụ của tôi! Chúc bạn mau khỏi bệnh."
            ]
        },
        # DISEASE DIAGNOSIS INTENTS
        {
            "tag": "symptom_fever",
            "patterns": [
                "tôi bị sốt", "sốt cao", "nhiệt độ cơ thể cao", "bị nóng người", "sốt và ớn lạnh",
                "sốt xuất huyết", "sốt virus", "sốt vi khuẩn", "sốt 39 độ", "sốt 40 độ",
                "fever", "high temperature", "feeling hot", "burning up", "chills and fever"
            ],
            "responses": [
                "🌡️ **TRIỆU CHỨNG: SỐT**\n\n**Nguyên nhân có thể:**\n- Nhiễm virus (cảm, cúm)\n- Nhiễm khuẩn\n- Sốt xuất huyết\n- Viêm nhiễm\n\n**Cần làm ngay:**\n✅ Đo nhiệt độ chính xác\n✅ Uống nhiều nước\n✅ Nghỉ ngơi\n✅ Dùng thuốc hạ sốt (Paracetamol)\n\n⚠️ **ĐẾN BỆNH VIỆN NGAY** nếu:\n- Sốt trên 39°C\n- Sốt kèm khó thở\n- Sốt kèm phát ban\n- Sốt kéo dài > 3 ngày\n\nBạn có muốn đặt lịch khám không?"
            ]
        },
        {
            "tag": "symptom_headache",
            "patterns": [
                "đau đầu", "nhức đầu", "đau nửa đầu", "migraine", "headache", "đau đầu dữ dội",
                "chóng mặt", "đau đầu kèm buồn nôn", "đau thái dương", "đau đầu mãn tính",
                "tension headache", "cluster headache", "đau đầu căng thẳng"
            ],
            "responses": [
                "🧠 **TRIỆU CHỨNG: ĐAU ĐẦU**\n\n**Phân loại:**\n- Đau đầu căng thẳng (phổ biến nhất)\n- Migraine (đau nửa đầu)\n- Đau đầu do sinus\n- Đau đầu cluster\n\n**Điều trị tại nhà:**\n✅ Nghỉ ngơi trong phòng tối\n✅ Chườm lạnh lên trán\n✅ Massage nhẹ thái dương\n✅ Uống đủ nước\n✅ Dùng thuốc giảm đau (Ibuprofen, Paracetamol)\n\n⚠️ **KHẨN CẤP** nếu:\n- Đau đầu đột ngột, dữ dội\n- Kèm sốt cao, cứng gáy\n- Kèm rối loạn thị giác\n- Kèm yếu liệt tay chân\n\nBạn muốn đặt lịch khám thần kinh không?"
            ]
        },
        {
            "tag": "symptom_cough",
            "patterns": [
                "ho", "ho khan", "ho có đờm", "ho ra máu", "cough", "ho kéo dài",
                "ho đêm", "ho sặc", "ho khò khè", "ho có đờm vàng", "ho có đờm xanh",
                "persistent cough", "dry cough", "wet cough", "whooping cough"
            ],
            "responses": [
                "🫁 **TRIỆU CHỨNG: HO**\n\n**Phân loại:**\n- Ho khan: virus, dị ứng, GERD\n- Ho có đờm: nhiễm khuẩn, viêm phổi\n- Ho ra máu: TB, ung thư phổi, nhiễm trùng\n\n**Điều trị:**\n✅ Uống nhiều nước ấm\n✅ Súc họng nước muối\n✅ Dùng mật ong chanh\n✅ Hít hơi nước ấm\n✅ Thuốc ho (Dextromethorphan)\n\n⚠️ **ĐẾN BỆNH VIỆN NGAY** nếu:\n- Ho ra máu\n- Khó thở\n- Sốt cao kèm ho\n- Ho > 2 tuần\n- Đau ngực khi ho\n\nCần đặt lịch khám phổi không?"
            ]
        },
        {
            "tag": "symptom_stomach_pain",
            "patterns": [
                "đau bụng", "đau dạ dày", "đau ruột", "stomach pain", "abdominal pain",
                "đau bụng dưới", "đau bụng trên", "đau quặn bụng", "đau bụng kèm nôn",
                "gastritis", "peptic ulcer", "appendicitis", "viêm ruột thừa", "viêm dạ dày"
            ],
            "responses": [
                "🤢 **TRIỆU CHỨNG: ĐAU BỤNG**\n\n**Vị trí & Nguyên nhân:**\n- Đau thượng vị: dạ dày, tá tràng\n- Đau hạ sườn phải: gan, túi mật\n- Đau hố chậu phải: ruột thừa\n- Đau toàn bụng: viêm phúc mạc\n\n**Điều trị:**\n✅ Nhịn ăn tạm thời\n✅ Uống nước nhỏ giọt\n✅ Nằm nghỉ, nén ấm\n✅ Tránh thuốc giảm đau\n\n🚨 **KHẨN CẤP** khi:\n- Đau dữ dội đột ngột\n- Cứng bụng\n- Nôn ra máu\n- Sốt cao kèm đau bụng\n- Đại tiện ra máu\n\nCần khám tiêu hóa ngay không?"
            ]
        },
        # APPOINTMENT BOOKING
        {
            "tag": "appointment_booking",
            "patterns": [
                "đặt lịch khám", "book appointment", "schedule appointment", "đặt hẹn",
                "muốn khám bệnh", "đăng ký khám", "tôi muốn đặt lịch", "làm sao để đặt lịch",
                "đặt lịch hẹn", "make appointment", "want to see doctor", "cần gặp bác sĩ"
            ],
            "responses": [
                "📅 **ĐẶT LỊCH KHÁM**\n\nTôi sẽ giúp bạn đặt lịch khám. Vui lòng cung cấp:\n\n1️⃣ **Chuyên khoa:** (tim mạch, da liễu, nhi, thần kinh...)\n2️⃣ **Thời gian:** (ngày mai, thứ 2, 15/12...)\n3️⃣ **Giờ ưu tiên:** (sáng, chiều, tối)\n\n💡 **Chuyên khoa có sẵn:**\n- 🫀 Tim mạch\n- 🧠 Thần kinh\n- 👶 Nhi khoa\n- 🦴 Xương khớp\n- 👁️ Mắt\n- 👂 Tai mũi họng\n- 🧴 Da liễu\n\nHãy cho tôi biết bạn muốn khám chuyên khoa gì?"
            ]
        },
        {
            "tag": "appointment_check",
            "patterns": [
                "kiểm tra lịch hẹn", "check appointment", "xem lịch khám", "tra lịch",
                "lịch hẹn của tôi", "my appointments", "khi nào tôi khám", "lịch khám bệnh",
                "appointment status", "view appointments", "see my schedule"
            ],
            "responses": [
                "📋 **KIỂM TRA LỊCH HẸN**\n\nĐể tra cứu lịch hẹn, tôi cần:\n\n🆔 **Số điện thoại** hoặc **Mã bệnh nhân**\n📧 **Email** đăng ký\n\n📱 **Hoặc bạn có thể:**\n- Đăng nhập vào app\n- Gọi hotline: 1900-XXX-XXX\n- Truy cập website bệnh viện\n\n💡 **Thông tin lịch hẹn bao gồm:**\n- Ngày giờ khám\n- Bác sĩ khám\n- Phòng khám\n- Ghi chú đặc biệt\n\nBạn có số điện thoại đăng ký không?"
            ]
        },
        # EMERGENCY DETECTION
        {
            "tag": "emergency",
            "patterns": [
                "cấp cứu", "khẩn cấp", "emergency", "help me", "tôi cần cấp cứu",
                "đau tim", "heart attack", "khó thở", "difficulty breathing", "stroke",
                "đột quỵ", "bất tỉnh", "unconscious", "chảy máu nhiều", "severe bleeding",
                "ngộ độc", "poisoning", "tai nạn", "accident", "gãy xương", "broken bone"
            ],
            "responses": [
                "🚨 **TÌNH HUỐNG KHẨN CẤP**\n\n**HÀNH ĐỘNG NGAY:**\n📞 **GỌI 115** (Cấp cứu)\n📞 **GỌI 114** (Cảnh sát)\n📞 **GỌI 113** (Cháy nổ)\n\n🏥 **BẤM SỐ HOTLINE:**\n- Bệnh viện Chợ Rẫy: (028) 3855 4269\n- Bệnh viện Bạch Mai: (024) 3869 3731\n- Bệnh viện E: (024) 3869 0736\n\n⚡ **CẤP CứU TẠI CHỖ:**\n✅ Giữ bình tĩnh\n✅ Đảm bảo an toàn\n✅ Sơ cứu cơ bản\n✅ Chờ xe cứu thương\n\n**KHÔNG TỰ Ý DI CHUYỂN** nếu nghi ngờ chấn thương cột sống!"
            ]
        },
        # HEALTH EDUCATION
        {
            "tag": "health_tips",
            "patterns": [
                "lời khuyên sức khỏe", "health tips", "cách chăm sóc sức khỏe",
                "sống khỏe", "healthy living", "phòng bệnh", "prevention",
                "tăng cường sức khỏe", "boost immunity", "tips sức khỏe"
            ],
            "responses": [
                "💡 **LỜI KHUYÊN SỐNG KHỎE**\n\n🥗 **DINH DƯỠNG:**\n- Ăn 5 phần rau củ/ngày\n- Uống 2-3L nước/ngày\n- Hạn chế đường, muối\n- Omega-3 từ cá biển\n\n🏃 **VẬN ĐỘNG:**\n- 150 phút/tuần cường độ vừa\n- Bơi lội, đi bộ, yoga\n- Tập thể dục buổi sáng\n\n😴 **NGHỈ NGƠI:**\n- Ngủ 7-9 tiếng/đêm\n- Đi ngủ trước 23h\n- Tránh màn hình trước ngủ\n\n🧘 **TINH THẦN:**\n- Thiền định, thư giãn\n- Kết nối xã hội\n- Quản lý stress\n\n🏥 **KHÁM ĐỊNH KỲ:**\n- Tổng quát 6 tháng/lần\n- Xét nghiệm máu hàng năm\n- Tầm soát ung thư"
            ]
        },
        {
            "tag": "medicine_info",
            "patterns": [
                "thông tin thuốc", "medicine information", "cách dùng thuốc",
                "tác dụng phụ", "side effects", "liều dùng", "dosage",
                "thuốc an toàn", "drug safety", "tương tác thuốc"
            ],
            "responses": [
                "💊 **THÔNG TIN THUỐC**\n\n⚠️ **NGUYÊN TẮC AN TOÀN:**\n- Chỉ dùng theo đơn bác sĩ\n- Đọc kỹ hướng dẫn\n- Không tự ý tăng liều\n- Báo bác sĩ về dị ứng\n\n📋 **THÔNG TIN CẦN BIẾT:**\n- Tên thuốc (hoạt chất)\n- Liều lượng & cách dùng\n- Thời gian điều trị\n- Tác dụng phụ\n- Tương tác thuốc\n\n🔍 **TRA CỨU THUỐC:**\n- Website Cục Dược\n- App thuốc chính thức\n- Hỏi dược sĩ\n\n⚡ **KHẨN CẤP:** Gọi 115 nếu nghi ngộ độc thuốc!\n\nBạn cần thông tin thuốc cụ thể nào?"
            ]
        }
    ]
}

print(f"✅ Created knowledge base with {len(healthcare_knowledge_base['intents'])} intents")
print("📋 Available intents:", [intent['tag'] for intent in healthcare_knowledge_base['intents']])